In [1]:
pwd

'/Users/brent/Documents/docs-shufe/ml_basic/proj/nbs'

In [1]:
data = '../data/previous_application.csv'

In [2]:
f_train, f_test = '../data/application_train.csv', '../data/application_test.csv'
f_train_id, f_test_id = '../data/train_sk_id_curr.txt', '../data/test_sk_id_curr.txt'

In [33]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
pd.set_option("display.max_columns", None)
@np.vectorize
def l1_normalize(val, _max, _min):
  return (val - _min)/(_max - _min + 1e-2)

In [4]:
df_main = pd.read_csv(data, dtype={"SK_ID_CURR":str, "SK_ID_PREV":str})

In [13]:
print(df_main.shape)
print(df_main.head(5))

(1670214, 37)
  SK_ID_PREV SK_ID_CURR NAME_CONTRACT_TYPE  AMT_ANNUITY  AMT_APPLICATION  \
0    2030495     271877     Consumer loans     1730.430          17145.0   
1    2802425     108129         Cash loans    25188.615         607500.0   
2    2523466     122040         Cash loans    15060.735         112500.0   
3    2819243     176158         Cash loans    47041.335         450000.0   
4    1784265     202054         Cash loans    31924.395         337500.0   

   AMT_CREDIT  AMT_DOWN_PAYMENT  AMT_GOODS_PRICE WEEKDAY_APPR_PROCESS_START  \
0     17145.0               0.0          17145.0                   SATURDAY   
1    679671.0               NaN         607500.0                   THURSDAY   
2    136444.5               NaN         112500.0                    TUESDAY   
3    470790.0               NaN         450000.0                     MONDAY   
4    404055.0               NaN         337500.0                   THURSDAY   

   HOUR_APPR_PROCESS_START FLAG_LAST_APPL_PER_CONTRACT

In [21]:
df_main.dtypes.sort_values()

NFLAG_LAST_APPL_IN_DAY           int64
SELLERPLACE_AREA                 int64
DAYS_DECISION                    int64
HOUR_APPR_PROCESS_START          int64
CNT_PAYMENT                    float64
DAYS_TERMINATION               float64
RATE_INTEREST_PRIVILEGED       float64
RATE_INTEREST_PRIMARY          float64
RATE_DOWN_PAYMENT              float64
DAYS_FIRST_DRAWING             float64
NFLAG_INSURED_ON_APPROVAL      float64
AMT_GOODS_PRICE                float64
AMT_DOWN_PAYMENT               float64
AMT_CREDIT                     float64
AMT_APPLICATION                float64
AMT_ANNUITY                    float64
DAYS_LAST_DUE_1ST_VERSION      float64
DAYS_LAST_DUE                  float64
DAYS_FIRST_DUE                 float64
CHANNEL_TYPE                    object
PRODUCT_COMBINATION             object
NAME_YIELD_GROUP                object
NAME_SELLER_INDUSTRY            object
NAME_PRODUCT_TYPE               object
SK_ID_PREV                      object
NAME_GOODS_CATEGORY      

In [9]:
# add bureau balance
df = df_main.copy()

## Remark

- SK_ID_CURR is the main-key, SK_ID_BUREAU can be agg.

In [22]:
cols_key_all = ["SK_ID_CURR", "SK_ID_PREV"]
cols_key_main = ["SK_ID_CURR"]
cols_ratio = []
# categoricals
cols_cate = [k for k, v in df_main.dtypes.items() if v.__str__() == 'object' and k not in cols_key_all]
# anything else is numerical (regular)
# numerics
cols_numeric = [k for k in df.columns if k not in cols_key_all and k not in cols_cate]

In [39]:
df_proc = df.copy().fillna(dict.fromkeys(cols_numeric, 0))

# one-hot encodings
cols_one_hot = []

for col in tqdm(cols_cate):
  _df_col = pd.get_dummies(df[col], prefix='ft')
  # print(_df_col.head(5))
  for feat_col in _df_col.columns:
    df_proc[feat_col] = _df_col[feat_col]
    cols_one_hot.append(feat_col)

print(f"categorical encodings finished #{len(cols_cate)} columns")

# do aggregation
df_agg = df_proc.groupby(cols_key_main)

numeric_methods = {"mean", "median", "max", "min"}
feature_series = {} 
columns_to_agg = cols_numeric + cols_one_hot + cols_ratio
for col in tqdm(columns_to_agg):
  for method in numeric_methods:
    feature_series[f"{col}_{method}"] = df_agg[col].agg(method)

df_final = pd.DataFrame(feature_series) 
for col in tqdm(df_final.columns):
  _max, _min = df_final[col].max(), df_final[col].min()
  df_final[col] = l1_normalize(df_final[col], _max, _min)


categorical encodings finished #16 columns


/Users/brent/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in ubyte_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


ZeroDivisionError: division by zero

In [42]:
df_final

,AMT_ANNUITY_min,AMT_ANNUITY_mean,AMT_ANNUITY_median,AMT_ANNUITY_max,AMT_APPLICATION_min,AMT_APPLICATION_mean,AMT_APPLICATION_median,AMT_APPLICATION_max,AMT_CREDIT_min,AMT_CREDIT_mean,AMT_CREDIT_median,AMT_CREDIT_max,AMT_DOWN_PAYMENT_min,AMT_DOWN_PAYMENT_mean,AMT_DOWN_PAYMENT_median,AMT_DOWN_PAYMENT_max,AMT_GOODS_PRICE_min,AMT_GOODS_PRICE_mean,AMT_GOODS_PRICE_median,AMT_GOODS_PRICE_max,HOUR_APPR_PROCESS_START_min,HOUR_APPR_PROCESS_START_mean,HOUR_APPR_PROCESS_START_median,HOUR_APPR_PROCESS_START_max,NFLAG_LAST_APPL_IN_DAY_min,NFLAG_LAST_APPL_IN_DAY_mean,NFLAG_LAST_APPL_IN_DAY_median,NFLAG_LAST_APPL_IN_DAY_max,RATE_DOWN_PAYMENT_min,RATE_DOWN_PAYMENT_mean,RATE_DOWN_PAYMENT_median,RATE_DOWN_PAYMENT_max,RATE_INTEREST_PRIMARY_min,RATE_INTEREST_PRIMARY_mean,RATE_INTEREST_PRIMARY_median,RATE_INTEREST_PRIMARY_max,RATE_INTEREST_PRIVILEGED_min,RATE_INTEREST_PRIVILEGED_mean,RATE_INTEREST_PRIVILEGED_median,RATE_INTEREST_PRIVILEGED_max,DAYS_DECISION_min,DAYS_DECISION_mean,DAYS_DECISION_median,DAYS_DECISION_max,SELLERPLACE_AREA_min,SELLERPLACE_AREA_mean,SELLERPLACE_AREA_median,SELLERPLACE_AREA_max,CNT_PAYMENT_min,CNT_PAYMENT_mean,CNT_PAYMENT_median,CNT_PAYMENT_max,DAYS_FIRST_DRAWING_min,DAYS_FIRST_DRAWING_mean,DAYS_FIRST_DRAWING_median,DAYS_FIRST_DRAWING_max,DAYS_FIRST_DUE_min,DAYS_FIRST_DUE_mean,DAYS_FIRST_DUE_median,DAYS_FIRST_DUE_max,DAYS_LAST_DUE_1ST_VERSION_min,DAYS_LAST_DUE_1ST_VERSION_mean,DAYS_LAST_DUE_1ST_VERSION_median,DAYS_LAST_DUE_1ST_VERSION_max,DAYS_LAST_DUE_min,DAYS_LAST_DUE_mean,DAYS_LAST_DUE_median,DAYS_LAST_DUE_max,DAYS_TERMINATION_min,DAYS_TERMINATION_mean,DAYS_TERMINATION_median,DAYS_TERMINATION_max,NFLAG_INSURED_ON_APPROVAL_min,NFLAG_INSURED_ON_APPROVAL_mean,NFLAG_INSURED_ON_APPROVAL_median,NFLAG_INSURED_ON_APPROVAL_max,ft_Cash loans_min,ft_Cash loans_mean,ft_Cash loans_median,ft_Cash loans_max,ft_Consumer loans_min,ft_Consumer loans_mean,ft_Consumer loans_median,ft_Consumer loans_max,ft_Revolving loans_min,ft_Revolving loans_mean,ft_Revolving loans_median,ft_Revolving loans_max,ft_XNA_min,ft_XNA_mean,ft_XNA_median,ft_XNA_max,ft_FRIDAY_min,ft_FRIDAY_mean,ft_FRIDAY_median,ft_FRIDAY_max,ft_MONDAY_min,ft_MONDAY_mean,ft_MONDAY_median,ft_MONDAY_max,ft_SATURDAY_min,ft_SATURDAY_mean,ft_SATURDAY_median,ft_SATURDAY_max,ft_SUNDAY_min,ft_SUNDAY_mean,ft_SUNDAY_median,ft_SUNDAY_max,ft_THURSDAY_min,ft_THURSDAY_mean,ft_THURSDAY_median,ft_THURSDAY_max,ft_TUESDAY_min,ft_TUESDAY_mean,ft_TUESDAY_median,ft_TUESDAY_max,ft_WEDNESDAY_min,ft_WEDNESDAY_mean,ft_WEDNESDAY_median,ft_WEDNESDAY_max,ft_N_min,ft_N_mean,ft_N_median,ft_N_max,ft_Y_min,ft_Y_mean,ft_Y_median,ft_Y_max,ft_Building a house or an annex_min,ft_Building a house or an annex_mean,ft_Building a house or an annex_median,ft_Building a house or an annex_max,ft_Business development_min,ft_Business development_mean,ft_Business development_median,ft_Business development_max,ft_Buying a garage_min,ft_Buying a garage_mean,ft_Buying a garage_median,ft_Buying a garage_max,ft_Buying a holiday home / land_min,ft_Buying a holiday home / land_mean,ft_Buying a holiday home / land_median,ft_Buying a holiday home / land_max,ft_Buying a home_min,ft_Buying a home_mean,ft_Buying a home_median,ft_Buying a home_max,ft_Buying a new car_min,ft_Buying a new car_mean,ft_Buying a new car_median,ft_Buying a new car_max,ft_Buying a used car_min,ft_Buying a used car_mean,ft_Buying a used car_median,ft_Buying a used car_max,ft_Car repairs_min,ft_Car repairs_mean,ft_Car repairs_median,ft_Car repairs_max,ft_Education_min,ft_Education_mean,ft_Education_median,ft_Education_max,ft_Everyday expenses_min,ft_Everyday expenses_mean,ft_Everyday expenses_median,ft_Everyday expenses_max,ft_Furniture_min,ft_Furniture_mean,ft_Furniture_median,ft_Furniture_max,ft_Gasification / water supply_min,ft_Gasification / water supply_mean,ft_Gasification / water supply_median,ft_Gasification / water supply_max,ft_Hobby_min,ft_Hobby_mean,ft_Hobby_median,ft_Hobby_max,ft_Journey_min,ft_Journey_mean,ft_Journey_median,ft_Journey_max,ft_Medi

In [43]:
df_final.to_csv("../data/proc/ft_prev_app.csv")

In [ ]:
!head -5  ../data/proc/ft_bureau.csv | tail -4